In [1]:
from datetime import datetime, timezone

from dateutil.relativedelta import relativedelta

from github_analysis.github.client.users import PullRequestStates
from github_analysis.github.service import GitHubService

PAGINATION_STEP_AMOUNT = 100
FILTER_STATES: list[PullRequestStates] = ["MERGED"]
NOW = datetime.now(tz=timezone.utc)
ONE_YEAR_AGO = NOW - relativedelta(years=1)
AGGREGATION_TIME = ONE_YEAR_AGO
USERNAME = "kentcdodds"

github_service = GitHubService()

(
    pull_request_reviews,
    merged_pull_requests,
) = await github_service.get_users_reviews_and_pull_requests(
    username=USERNAME,
    pagination_step_amount=PAGINATION_STEP_AMOUNT,
    from_date=AGGREGATION_TIME,
    pull_request_states=FILTER_STATES,
)

In [2]:
from github_analysis.data_frame_processing import (
    group_pull_requests_by_repositories,
    pull_requests_as_data_frame,
)

merged_pull_requests_data_frame = pull_requests_as_data_frame(
    pull_requests=merged_pull_requests
)
grouped_merged_pull_requests = group_pull_requests_by_repositories(
    pull_requests=merged_pull_requests_data_frame
)
grouped_merged_pull_requests

repository_name,amount_of_pull_requests
str,u32
"""skillrecordings/products""",20
"""epicweb-dev/testing-fundamenta…",5
"""epicweb-dev/pixel-perfect-tail…",5
"""epicweb-dev/epic-stack""",3
"""remix-run/remix""",2
…,…
"""kentcdodds/kentcdodds.com""",1
"""reactjs/react.dev""",1
"""epicweb-dev/mocking-techniques""",1


In [3]:
from github_analysis.data_frame_processing import (
    group_pull_request_reviews_by_repositories,
    pull_request_reviews_as_data_frame,
)

pull_request_reviews_data_frame = pull_request_reviews_as_data_frame(
    pull_request_reviews=pull_request_reviews
)
grouped_pull_request_reviews = group_pull_request_reviews_by_repositories(
    pull_request_reviews=pull_request_reviews_data_frame
)
grouped_pull_request_reviews

repository_name,amount_of_reviews
str,u32
"""epicweb-dev/epic-stack""",96
"""epicweb-dev/full-stack-foundat…",40
"""epicweb-dev/data-modeling""",34
"""epicweb-dev/web-auth""",31
"""kentcdodds/kentcdodds.com""",29
…,…
"""epicweb-dev/pixel-perfect-tail…",1
"""kentcdodds/advanced-react-patt…",1
"""epicweb-dev/remember""",1


In [4]:
from github_analysis.data_frame_processing import process_total_stats

total_stats = process_total_stats(
    grouped_pull_request_reviews=grouped_pull_request_reviews,
    grouped_merged_pull_requests=grouped_merged_pull_requests,
)
total_stats

stat,totals
str,i64
"""reviews""",358
"""pull_requests""",49
